In [ ]:
## COMPUTE Single point energy and thermo properties using Auto3D and AIMNet2.ase.calculator

In [3]:
import Auto3D
from Auto3D.auto3D import options, main
from Auto3D.SPE import calc_spe
from Auto3D.ASE.thermo import calc_thermo
from ase.io import read
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import Vibrations
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton
from aimnet2calc import AIMNet2ASE
from ase.optimize import FIRE
from ase.optimize import BFGS

In [2]:
import os
os.chdir("/mnt/c/Users/user/Downloads/orca_sp_large_systems/quinone/radical")

In [5]:
path='14_Q_0.sdf'
spe = calc_spe(path, model_name="AIMNET")

In [8]:


# Step 1: Read your molecule
atoms = read('14_Q_24.sdf')

# Step 2: Assign a calculator suitable for organics
# from xtb.ase.calculator import XTB
# atoms.calc = XTB(method='GFN2-xTB')

calc = AIMNet2ASE('aimnet2', charge=0)
atoms.calc = calc

# atoms.calc = EMT()
# dyn = QuasiNewton(atoms)
# dyn.run(fmax=0.01)

# atoms.calc = calc

# Step 3: Perform geometry optimization
# dyn = QuasiNewton(atoms)
# dyn.run(fmax=0.01)

# dyn = FIRE(atoms)
# dyn.run(fmax=0.0001, steps=2000)

dyn = BFGS(atoms)
dyn.run(fmax=0.001)

# Step 4: Perform vibrational analysis
# vib.clean() 
vib = Vibrations(atoms)
vib.run()

# Step 5: Get vibrational energies
vib_energies = vib.get_energies()

potentialenergy = atoms.get_potential_energy()

# Step 6: Thermochemical calculations
thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='nonlinear',  # Adjust if your molecule is linear
                        symmetrynumber=1,      # Adjust based on your molecule's symmetry
                        spin=0)
                        # ignore_imag_modes=True)

# Compute Gibbs free energy at specified temperature and pressure
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)

print("Gibbs free energy (in eV):", G)
##WORKS!!

Found model file: /home/ilkwon/anaconda3/envs/Comp_lecture/lib/python3.10/site-packages/aimnet2calc-0.0.1-py3.10.egg/aimnet2calc/assets/aimnet2/aimnet2_wb97m_0.jpt
      Step     Time          Energy          fmax
BFGS:    0 17:11:54   -16678.745070        0.080702
BFGS:    1 17:11:54   -16678.745342        0.066517
BFGS:    2 17:11:54   -16678.745578        0.039941
BFGS:    3 17:11:54   -16678.745797        0.040468
BFGS:    4 17:11:54   -16678.745872        0.031218
BFGS:    5 17:11:54   -16678.745919        0.022827
BFGS:    6 17:11:54   -16678.745953        0.013777
BFGS:    7 17:11:54   -16678.745976        0.012134
BFGS:    8 17:11:54   -16678.745986        0.007361
BFGS:    9 17:11:54   -16678.745993        0.006691
BFGS:   10 17:11:54   -16678.745999        0.006137
BFGS:   11 17:11:54   -16678.746007        0.008831
BFGS:   12 17:11:54   -16678.746008        0.004910
BFGS:   13 17:11:54   -16678.746013        0.004197
BFGS:   14 17:11:54   -16678.746012        0.002714
BFGS: 

In [12]:
import os 
import h5py
from glob import glob
import numpy as np
from openbabel import openbabel as ob 
import torch
import ase
from ase.io import read, write
from ase.calculators.orca import ORCA
from ase.calculators.orca import OrcaProfile
from ase import Atoms
from openbabel import pybel
import requests
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import shutil
from rdkit.Chem import rdmolops
from tqdm import tqdm
import subprocess
from mace.calculators import mace_off
from numpy import printoptions

# filename='/mnt/c/Users/user/Downloads/pergamonn_0710/pergamonn-main/pergamonn-main/aimnet_models/aimnet2_b973c_ens.jpt'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = torch.jit.load(filename, map_location=device)

hartree2ev = 27.211386245988
ev2hartree = 1/hartree2ev
ev2kcalmol = 23.0605

# profile = OrcaProfile(command='/mnt/c/Users/user/Downloads/orca_6_0_0_shared_openmpi416/orca')
model_path_b973c='/mnt/c/Users/user/Downloads/pergamonn_0710/pergamonn-main/pergamonn-main/aimnet_models/aimnet2_b973c_ens.jpt'
model_path_wb97m='/mnt/c/Users/user/Downloads/pergamonn_0710/pergamonn-main/pergamonn-main/aimnet_models/aimnet2_wb97m-d3_ens.jpt'
model_path_nse='/mnt/c/Users/user/Downloads/AIMNET2-NSE/aimnet2nse_240827_ens.jpt'

def AIMNET2_wb97(mol_path, charge, model_path=model_path_wb97m,ev2hartree=ev2hartree):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = torch.jit.load(model_path, map_location=device)

    # Read the molecule file using Open Babel
    mol = next(pybel.readfile('sdf', mol_path))

    # Prepare input tensors for the model
    coord = torch.as_tensor([a.coords for a in mol.atoms]).unsqueeze(0).to(device)
    coord.requires_grad_(True)

    numbers = torch.as_tensor([a.atomicnum for a in mol.atoms]).unsqueeze(0).to(device)
    charge = torch.as_tensor([charge]).to(device)


    _in = dict(
        coord=coord,
        numbers=numbers,
        charge=charge
    )

    _out = model(_in)

    energy_hartree = _out['energy'].item() * ev2hartree
    # print(energy_hartree)
    return energy_hartree


def AIMNET2_NSE(mol_path, charge, model_path=model_path_nse,ev2hartree=ev2hartree):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = torch.jit.load(model_path, map_location=device)

    # Read the molecule file using Open Babel
    mol = next(pybel.readfile('sdf', mol_path))

    # Prepare input tensors for the model
    coord = torch.as_tensor([a.coords for a in mol.atoms]).unsqueeze(0).to(device)
    coord.requires_grad_(True)

    numbers = torch.as_tensor([a.atomicnum for a in mol.atoms]).unsqueeze(0).to(device)
    charge = torch.as_tensor([charge]).to(device)
    mult=1
    mult = torch.as_tensor([mult]).to(device)

    _in = dict(
        coord=coord,
        numbers=numbers,
        charge=charge,
        mult=mult
    )

    _out = model(_in)

    energy_hartree = _out['energy'].item() * ev2hartree
    # print(energy_hartree)
    return energy_hartree

In [14]:
AIMNET2_NSE('14_Q_0.sdf',charge=0)

-381.71575000602274

In [1]:
import ipywidgets as widgets
import IPython.display as display
from IPython.display import Video
from IPython.display import HTML
import subprocess as sp
import os, sys

from urllib.request import urlopen
from io import StringIO
import shutil
from pathlib import Path

In [2]:
def fetch_pdb(pdbid):
  print("Downloading : http://www.rcsb.org/pdb/files/%s.pdb" % pdbid)
  url = 'http://www.rcsb.org/pdb/files/%s.pdb' % pdbid
  file = urlopen(url)
  contents = file.read().decode('utf-8')
  file.close()
  file = StringIO(contents)
  outfile=pdbid+".pdb"
  with open(outfile, 'w') as f2:
      for line in contents:
          f2.write(line)

In [5]:
pdbid="5QCZ"
fetch_pdb(pdbid)

In [7]:
cmd='pkaani -i '+pdbid
_o=!{cmd}

In [8]:
!pwd

/mnt/c/Users/user/Desktop/Isayev_LAB


In [4]:
class protonation:
      def __init__(self,pkaanilog):
          self.logfile=pkaanilog
          #self.amber_type=dict()
          
      def get_amber_res_type(self,residue,pka):
        # pka in water: 
        #   glu 4.5, 
        #   asp 3.8, 
        #   his 6.5, 
        #   lys 10.5, 
        #   tyr 10.0 --> no different 
        #                amber type so it will skip it
        # I want to use ~0.5 tolerance 
        # my models mse are below and about 0.5
        #but we can talk about it
        newres=residue   
        if(residue=='GLU' and pka>5.0):
           newres='GLH'
        if(residue=='ASP' and pka>4.3):
           newres='ASH'       
        if(residue=='HIS' and pka>7.0):
           newres='HIP'  
        if(residue=='LYS' and pka<9.0):
           # I want to be more relaxed for LYS 
           print(pka) 
           newres='LYN'  

        return newres   
          
      def decision(self):

        newres=dict()   
        with open(self.logfile, 'r+') as fd:
          contents = fd.readlines()[1:]
          #Residue	Chain	pKa
          #TYR-3	A	10.85  
          #first find if there is an SSBOND
          for line in contents:
              row=line.strip().split()
              resname=row[0].split('-')[0]
              resno=row[0].split('-')[1]
              chain=row[1]
              pka=row[2]

              restyp=self.get_amber_res_type(resname,float(pka))
              if(restyp!=resname):
                key=resname+'-'+chain+'-'+resno
                newres[key]=[restyp,float(pka)]
              

        return newres  

In [5]:
workdir=os.getcwd()

proteinResidues = ['ALA', 'ASN', 'CYS', 'GLU', 'HIS', 
                   'LEU', 'MET', 'PRO', 'THR', 'TYR', 
                   'ARG', 'ASP', 'GLN', 'GLY', 'ILE', 
                   'LYS', 'PHE', 'SER', 'TRP', 'VAL',
                   'HID', 'HIE', 'HIP', 'LYN', 'CYX',
                   'CYM','GLH','ASH']

In [6]:
interdir=Path(os.path.join(workdir,pdbid+"_intermediate_files"))
if interdir.exists() and interdir.is_dir():
    shutil.rmtree(interdir)
os.mkdir(interdir)

In [24]:
decider=protonation(pdbid+'_pka.log')
protonations=decider.decision()
print('%-10s %12s %6s'%("Res-Ch-Rno","Amber_name","pka"))
for key,values in protonations.items():
    print('%-10s %10s %8.2f'%(key,values[0],values[1]))

#clean up
if not os.path.exists('pkaani_results'): os.mkdir('pkaani_results')
srcf=os.path.join(workdir,pdbid+'_pka.log')
destf=os.path.join(workdir+"/pkaani_results",pdbid+'_pka.log')
os.rename(srcf,destf)

srcf=os.path.join(workdir,pdbid+'_pkaani.pdb')
destf=os.path.join(workdir+"/pkaani_results",pdbid+'_pkaani.pdb')
os.rename(srcf,destf)

Res-Ch-Rno   Amber_name    pka


In [23]:
## with potential energy


atoms = molecule('N2')

atoms.calc = EMT()
dyn = QuasiNewton(atoms)
dyn.run(fmax=0.01)

potentialenergy = atoms.get_potential_energy()

vib = Vibrations(atoms)
vib.run()
vib_energies = vib.get_energies()

thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='linear',
                        symmetrynumber=2, spin=0)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.)

print("Gibbs energy:", G)


Enthalpy components at T = 298.15 K:
E_pot                  0.549 eV
E_ZPE                  0.076 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.026 eV
Cv_vib (0->T)          0.000 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                      0.715 eV

Entropy components at T = 298.15 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0015590 eV/K        0.465 eV
S_rot              0.0004314 eV/K        0.129 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0000016 eV/K        0.000 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0019909 eV/K        0.594 eV

Free energy components at T = 298.15 K and P = 101325.0 Pa:
    H          0.715 eV
 -T*S         -0.594 eV
-----------------------
    G          0.122 eV
Gibbs energy: 0.12182464017289352


In [59]:
from ase.optimize import BFGS
from ase.io import read
from ase.calculators.emt import EMT
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.visualize import view 

# Read the SDF file containing the molecular geometry
path='/mnt/c/Users/user/Desktop/Isayev_LAB/Hati_Project/AIMNET_QB_0121/12A/jacs/tyk2/cutoff4/tyk_fmax_0p5_rcut_4_b973c_substruct'

os.chdir(path)


atoms = read('tyk2_ligands_jmc_30_min.sdf')


# Set up the calculator (EMT for example)

# atoms.calc = EMT() -> x for cl2

lj_params = {'Cl': {'epsilon': 0.1, 'sigma': 3.5}}  
atoms.calc = LennardJones(**lj_params)

# opt = BFGS(atoms)
# opt.run(fmax=0.001) 

# Calculate potential energy
potentialenergy = atoms.get_potential_energy()

# Calculate vibrational frequencies
vib = Vibrations(atoms)
vib.run()
vib_energies = vib.get_energies()


# # Compute thermodynamic properties
thermo = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=atoms,
                        geometry='nonlinear',  # Adjust as needed
                        symmetrynumber=1, spin=0)  # Adjust as needed


# entropy = thermo.get_entropy(temperature=298.15, pressure=101325.)
# # Print the Gibbs energy
# print("Total Entropy:", entropy)


ValueError: Imaginary frequencies are present.